In [1]:
# If you want to execute the notebook on colab, uncommet the following line.
#! git clone https://github.com/TimotheeMathieu/MOM_NN

import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam,RMSprop, Nadam

from tensorflow.keras.utils import to_categorical
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [2]:
print(X_train.shape)
# Flatten the data
X_train = X_train.reshape([X_train.shape[0], 784])
X_test = X_test.reshape([X_test.shape[0], 784])

(60000, 28, 28)


In [3]:
# Add 50 outliers randomly
X_train[np.random.permutation(len(X_train))[:50], :] = 0 # corresponds to an all black image

In [4]:
# one hot encoding
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [5]:
print(X_train.shape, y_train.shape)

(60000, 784) (60000, 10)


In [6]:
# Use dense network for simplification, a convnet would have been better.
input_layer = Input(shape=(len(X_train[0]), ))
encoder = Dense(32, activation="relu")(input_layer)
encoder = Dense(10, activation="softmax")(encoder)
model = Model(inputs=input_layer, outputs=encoder)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 32)                25120     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
from sklearn.metrics import roc_auc_score

In [9]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 6.4566 - accuracy: 0.3927
Epoch 2/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.8714 - accuracy: 0.7357
Epoch 3/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.5891 - accuracy: 0.8481
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4570 - accuracy: 0.8829
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3925 - accuracy: 0.8984
Epoch 6/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3539 - accuracy: 0.9073
Epoch 7/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3290 - accuracy: 0.9124
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3169 - accuracy: 0.9159
Epoch 9/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3210 - accuracy: 0.9140
Epoch 10/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.305

In [10]:
pred = model.predict(X_test).argmax(axis=1)
true_label  = y_test.argmax(axis=1)
print(np.mean(true_label == pred))

0.9075


# Use MOM

In [11]:
from keras_mom import MOM_model
from tensorflow.keras.losses import CategoricalCrossentropy, Reduction
# Initialize with the pre-trained model just used
model2 = MOM_model(model, CategoricalCrossentropy(reduction=Reduction.NONE), 
                   K=71, batch_size=64, max_iter=100, verbose=1)

In [12]:
model2.fit(X_train, y_train)
# code not really optimized but it computes. Because each epoch is only on sample_size/K points
# it needs more iterations to converge.

14/14 [==============================] - 0s 1ms/step - loss: 0.2839 - accuracy: 0.9290
Epoch 2/2
14/14 [==============================] - 0s 1ms/step - loss: 0.2837 - accuracy: 0.9231
Epoch 3/3
14/14 [==============================] - 0s 1ms/step - loss: 0.2833 - accuracy: 0.9219
Epoch 4/4
14/14 [==============================] - 0s 1ms/step - loss: 0.2910 - accuracy: 0.9314
Epoch 5/5
14/14 [==============================] - 0s 1ms/step - loss: 0.2738 - accuracy: 0.9172
Epoch 6/6
14/14 [==============================] - 0s 1ms/step - loss: 0.2757 - accuracy: 0.9112
Epoch 7/7
14/14 [==============================] - 0s 1ms/step - loss: 0.2612 - accuracy: 0.9349
Epoch 8/8
14/14 [==============================] - 0s 1ms/step - loss: 0.2570 - accuracy: 0.9278
Epoch 9/9
14/14 [==============================] - 0s 1ms/step - loss: 0.2602 - accuracy: 0.9231
Epoch 10/10
14/14 [==============================] - 0s 1ms/step - loss: 0.2669 - accuracy: 0.9373
Epoch 11/11
14/14 [===================

14/14 [==============================] - 0s 1ms/step - loss: 0.3041 - accuracy: 0.9183
Epoch 85/85
14/14 [==============================] - 0s 1ms/step - loss: 0.2708 - accuracy: 0.9302
Epoch 86/86
14/14 [==============================] - 0s 1ms/step - loss: 0.2860 - accuracy: 0.9148
Epoch 87/87
14/14 [==============================] - 0s 1ms/step - loss: 0.2821 - accuracy: 0.9266
Epoch 88/88
14/14 [==============================] - 0s 2ms/step - loss: 0.2627 - accuracy: 0.9314
Epoch 89/89
14/14 [==============================] - 0s 1ms/step - loss: 0.2527 - accuracy: 0.9266
Epoch 90/90
14/14 [==============================] - 0s 1ms/step - loss: 0.2491 - accuracy: 0.9337
Epoch 91/91
14/14 [==============================] - 0s 1ms/step - loss: 0.2592 - accuracy: 0.9231
Epoch 92/92
14/14 [==============================] - 0s 1ms/step - loss: 0.2573 - accuracy: 0.9325
Epoch 93/93
14/14 [==============================] - 0s 1ms/step - loss: 0.2642 - accuracy: 0.9149
Epoch 94/94
14/14 [===

/home/frost/info/venv/python38_venv/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


MOM_model(K=71, batch_size=64,
          loss=<tensorflow.python.keras.losses.CategoricalCrossentropy object at 0x7f597c338bb0>,
          model=<tensorflow.python.keras.engine.functional.Functional object at 0x7f5a3ca129d0>,
          verbose=1)

In [15]:
pred = model2.predict(X_test).argmax(axis=1)
true_label  = y_test.argmax(axis=1)
print(np.mean(true_label == pred))

0.9161


In [16]:
np.arange(0, 10,3)

array([0, 3, 6, 9])